In [30]:
!pip install seaborn
!pip install xgboost
!pip install --upgrade xgboost
!pip install catboost
!pip install pycaret #pycaret 설치
!pip install category_encoders
!pip install wandb


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import json
import pandas as pd
import numpy as np
from pycaret.classification import *
from preprocess import DataPreprocessor
import wandb

In [2]:
wandb.login()
wandb.init(project='automl')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ans2568 (ai-leaders). Use `wandb login --relogin` to force relogin


In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_test = df_test.drop(['id', 'Unnamed: 0'], axis=1)

In [5]:
df_all = pd.concat([df_train, df_test]).reset_index(drop=True)

In [6]:
with open('config.json') as f:
    preprocess_columns = json.load(f)

In [7]:
preprocessor = DataPreprocessor(preprocess_columns, train_mode=True)
df_train_processed = preprocessor.fit_transform(df_train)

In [8]:
preprocessor_test = DataPreprocessor(preprocess_columns, train_mode=False)
df_test_processed = preprocessor_test.transform(df_test)

In [9]:
model = setup(
    data = df_train_processed,
    train_size=0.8,
    normalize=True,
    normalize_method='minmax', 
    target = "is_converted",
    fold = 15,
    n_jobs=1,
    log_experiment='wandb',
)

,Description,Value
0,Session id,2532
1,Target,is_converted
2,Target type,Binary
3,Original data shape,"(59299, 25)"
4,Transformed data shape,"(59299, 25)"
5,Transformed train set shape,"(47439, 25)"
6,Transformed test set shape,"(11860, 25)"
7,Numeric features,23
8,Categorical features,1
9,Preprocess,True


In [10]:
model = compare_models(sort='F1', fold = 5, n_select=5) # top5만

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9763,0.9894,0.7794,0.9182,0.8430,0.8303,0.8336,0.5300
rf,Random Forest Classifier,0.9755,0.9876,0.7374,0.9518,0.8308,0.8178,0.8257,2.6620
lightgbm,Light Gradient Boosting Machine,0.9742,0.9882,0.7374,0.9338,0.8240,0.8103,0.8169,0.9120
et,Extra Trees Classifier,0.9718,0.9791,0.7139,0.9245,0.8056,0.7907,0.7984,2.0220
dt,Decision Tree Classifier,0.9670,0.8968,0.8108,0.7913,0.8008,0.7829,0.7830,0.2840
gbc,Gradient Boosting Classifier,0.9664,0.9767,0.6168,0.9580,0.7502,0.7332,0.7537,4.7360
ada,Ada Boost Classifier,0.9573,0.9681,0.5706,0.8601,0.6860,0.6641,0.6802,1.2820
knn,K Neighbors Classifier,0.9497,0.9034,0.5745,0.7522,0.6512,0.6247,0.6314,1.5060
qda,Quadratic Discriminant Analysis,0.8501,0.8401,0.5562,0.2925,0.3802,0.3057,0.3275,0.2360
nb,Naive Bayes,0.8383,0.7945,0.4276,0.2333,0.3017,0.2191,0.2321,0.1320


In [11]:
xgboost = create_model('xgboost')
final_model = finalize_model(xgboost)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9760,0.9891,0.7722,0.9217,0.8403,0.8275,0.8312
1,0.9810,0.9920,0.8340,0.9270,0.8780,0.8678,0.8692
2,0.9750,0.9894,0.7645,0.9167,0.8337,0.8203,0.8243
3,0.9753,0.9912,0.7683,0.9171,0.8361,0.8229,0.8267
4,0.9741,0.9903,0.7452,0.9234,0.8248,0.8110,0.8164
5,0.9763,0.9922,0.7954,0.9035,0.8460,0.8332,0.8352
6,0.9791,0.9912,0.8108,0.9251,0.8642,0.8529,0.8551
7,0.9791,0.9892,0.8031,0.9327,0.8631,0.8518,0.8546
8,0.9722,0.9883,0.7683,0.8767,0.8189,0.8039,0.8060


In [13]:
# 예측에 필요한 데이터 분리
x_test = df_test_processed.drop(["is_converted"], axis=1)

pred = predict_model(final_model, data=x_test)

In [16]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
# pred = pred.set_index(df_sub.index)
df_sub["is_converted"] = pred["prediction_label"].astype(bool)
df_sub

,Unnamed: 0,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,False
2,2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,False
5268,5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,False
5269,5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,False


In [18]:
# 제출 파일 저장
df_sub.to_csv("submission_experiment_base.csv", index=False)